# COVIDvu <img src='resources/UN-flag.png' align='right'>

COVID-19 view various charts.

---
## Runtime prerequisites

In [ ]:
%%capture --no-stderr requirementsOutput

displayRequirementsOutput = False

%pip install -r requirements.txt
from covidvu.utils import autoReloadCode; autoReloadCode()

In [ ]:
if displayRequirementsOutput:
    requirementsOutput.show()

---
## Pull latest datasets from the JH CSSE GitHub repository


In [ ]:
%sx ./refreshdata local patch

In [ ]:
!env | sort

---
## Confirmed, deaths, recovered datasets

In [ ]:
import os

import numpy as np
import pandas as pd

from covidvu.pipeline.vujson import parseCSSE

pd.options.mode.chained_assignment = None


In [ ]:
confirmedCases = parseCSSE('confirmed')['casesGlobal']
confirmedDeaths = parseCSSE('deaths')['casesGlobal']

In [ ]:
mortalityRate = confirmedDeaths/confirmedCases

---
## Interactive plots

In [ ]:
from covidvu import visualize
from ipywidgets import widgets
from ipywidgets import interact
from ipywidgets import fixed

In [ ]:
countries = list(confirmedCases.columns)
multiCountry = widgets.SelectMultiple(
    options=countries,
    value=['!Global', '!Outside China'],
    description='Country',
    disabled=False
)
log = widgets.Checkbox(value=False, description='Log scale')

### Confirmed cases

In [ ]:
interact(visualize.plotTimeSeriesInteractive, 
         df=fixed(confirmedCases), 
         selectedColumns=multiCountry, 
         log=log,
         yLabel=fixed('Total confirmed cases'),
         title=fixed('COVID-19 total confirmed cases')
);

In [ ]:
def viewTopCountries(n):
    sortedCountries = confirmedCases.iloc[-1,:].sort_values(ascending=False)
    sortedCountriesRaw = sortedCountries[sortedCountries.index.map(lambda c: c[0]!="!")]
    return pd.DataFrame(sortedCountriesRaw.iloc[:n]).style.background_gradient(cmap="Reds")

In [ ]:
interact(viewTopCountries, n=widgets.IntSlider(min=1, max=confirmedCases.shape[0], step=1, value=5));

### New cases

In [ ]:
interact(visualize.plotTimeSeriesInteractive, 
         df=fixed(confirmedCases.diff(axis=0)), 
         selectedColumns=multiCountry, 
         log=log,
         yLabel=fixed('Confirmed cases (per day)'),
         title=fixed('COVID-19 confirmed new cases (per day)')
);

### Confirmed deaths

In [ ]:
interact(visualize.plotTimeSeriesInteractive, 
         df=fixed(confirmedDeaths), 
         selectedColumns=multiCountry, 
         log=log,
         yLabel=fixed('Number of deaths'),
         title=fixed('COVID-19 total confirmed deaths')
);

---
## Mortality and recovery rates

### Mortality rate

In [ ]:
interact(visualize.plotTimeSeriesInteractive, 
         df=fixed(mortalityRate), 
         selectedColumns=multiCountry, 
         log=log,
         yLabel=fixed('Mortality rate (deaths/cases)'),
         title=fixed('COVID-19 mortality rate')
);

---
&#169; the COVIDvu Contributors.  All rights reserved.